<a href="https://colab.research.google.com/github/HudaKhraiss/ADS2002-Group-Project/blob/Colin's-Branch/ADS2002_Group_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import plotly.express as px
import plotly.io as pio
pio.templates
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from numpy import asarray
import os
from PIL import Image
import pandas as pd

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
df = pd.read_csv('/content/drive/My Drive/ranzcr-clip-catheter-line-classification/train.csv')

In [55]:
df

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,1,0,0,0,1,0,bf4c6da3c
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,0,1,0,0,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,0,0,1,0,0,0,0,0,1,1,0,5b5b9ac30
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,0,0,0,0,0,0,0,0,0,1,0,7192404d8
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,0,0,1,0,0,1,0,1,0,1,0,d4d1b066d
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,0,0,0,0,0,0,0,0,1,0,0,01a6602b8


In [56]:
annotations = pd.read_csv('/content/drive/My Drive/ranzcr-clip-catheter-line-classification/train_annotations.csv')

In [57]:
annotations.head()

,StudyInstanceUID,label,data
0,1.2.826.0.1.3680043.8.498.12616281126973421762...,CVC - Normal,"[[1487, 1279], [1477, 1168], [1472, 1052], [14..."
1,1.2.826.0.1.3680043.8.498.12616281126973421762...,CVC - Normal,"[[1328, 7], [1347, 101], [1383, 193], [1400, 2..."
2,1.2.826.0.1.3680043.8.498.72921907356394389969...,CVC - Borderline,"[[801, 1207], [812, 1112], [823, 1023], [842, ..."
3,1.2.826.0.1.3680043.8.498.11697104485452001927...,CVC - Normal,"[[1366, 961], [1411, 861], [1453, 751], [1508,..."
4,1.2.826.0.1.3680043.8.498.87704688663091069148...,NGT - Normal,"[[1862, 14], [1845, 293], [1801, 869], [1716, ..."


In [58]:
df.isna().sum()

StudyInstanceUID              0
ETT - Abnormal                0
ETT - Borderline              0
ETT - Normal                  0
NGT - Abnormal                0
NGT - Borderline              0
NGT - Incompletely Imaged     0
NGT - Normal                  0
CVC - Abnormal                0
CVC - Borderline              0
CVC - Normal                  0
Swan Ganz Catheter Present    0
PatientID                     0
dtype: int64

In [59]:
df.duplicated().sum()

0

In [60]:
df.dtypes

StudyInstanceUID              object
ETT - Abnormal                 int64
ETT - Borderline               int64
ETT - Normal                   int64
NGT - Abnormal                 int64
NGT - Borderline               int64
NGT - Incompletely Imaged      int64
NGT - Normal                   int64
CVC - Abnormal                 int64
CVC - Borderline               int64
CVC - Normal                   int64
Swan Ganz Catheter Present     int64
PatientID                     object
dtype: object

In [61]:
df.shape

(30083, 13)

In [62]:
df = df.drop("NGT - Incompletely Imaged", axis=1)

In [63]:
df = df.drop("Swan Ganz Catheter Present", axis=1)

In [64]:
# Create a new column 'num_ones' to count the number of 1s in each row
columns_to_exclude = ['StudyInstanceUID', 'PatientID']  # Add columns you want to exclude
df['num_ones'] = df.drop(columns_to_exclude, axis=1).sum(axis=1)

df

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,PatientID,num_ones
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,1,0,0,0,ec89415d1,1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,0,0,0,1,bf4c6da3c,2
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,1,0,3fc1c97e5,1
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,1,0,0,c31019814,1
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,1,207685cd1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,0,0,1,0,0,0,0,1,1,5b5b9ac30,3
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,0,0,0,0,0,0,0,0,1,7192404d8,1
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,0,0,1,0,0,0,1,0,1,d4d1b066d,3
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,0,0,0,0,0,0,0,1,0,01a6602b8,1


In [65]:
# Drop the rows where num_ones is greater than 1
df = df[df['num_ones'] <= 1]

# Drop the 'num_ones' column as it's no longer needed
df = df.drop('num_ones', axis=1)

# Print the filtered DataFrame
df

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,1,0,0,0,ec89415d1
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,1,0,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,1,0,0,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,1,207685cd1
8,1.2.826.0.1.3680043.8.498.91266853519846682206...,0,0,0,0,0,0,0,0,1,2edf3dce3
...,...,...,...,...,...,...,...,...,...,...,...
30075,1.2.826.0.1.3680043.8.498.12112840402677606176...,0,0,0,0,0,0,0,1,0,cccbc15ba
30076,1.2.826.0.1.3680043.8.498.59704742952729813362...,0,0,0,0,0,0,1,0,0,172c3c7ed
30077,1.2.826.0.1.3680043.8.498.97304417279653947772...,0,0,0,0,0,0,0,1,0,b304abf90
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,0,0,0,0,0,0,0,0,1,7192404d8


In [66]:
df.shape

(19240, 11)

In [67]:
print("Number of unique patients: ", df["PatientID"].unique().shape[0])

Number of unique patients:  3126


In [68]:
idCounts = df['PatientID'].value_counts().reset_index()
idCounts.columns = ['PatientID', 'Number of Observations']
idCounts = idCounts.sort_values(by = 'Number of Observations', ascending = False)
idCounts

,PatientID,Number of Observations
0,b877e913c,63
1,f93cf7e67,55
2,bb76dda43,52
3,e8f177830,50
4,d01de7ac5,47
...,...,...
2785,31e224370,1
2784,c0b57c3d9,1
2783,7755053cb,1
2782,e960df13c,1


In [69]:
fig = px.histogram(idCounts, x='Number of Observations', title='Distribution of Number of Observations per PatientIDs', template='plotly_white')
fig.show()

In [70]:
categories = ['ETT - Abnormal',
              'ETT - Borderline',
              'ETT - Normal',
              'NGT - Abnormal',
              'NGT - Borderline',
              'NGT - Normal',
              'CVC - Abnormal',
              'CVC - Borderline',
              'CVC - Normal']

categoryCounts = df[categories].sum(axis = 0).reset_index()
categoryCounts.columns = ['Malpositions', 'Number of Observations']
categoryCounts

,Malpositions,Number of Observations
0,ETT - Abnormal,3
1,ETT - Borderline,25
2,ETT - Normal,195
3,NGT - Abnormal,17
4,NGT - Borderline,25
5,NGT - Normal,97
6,CVC - Abnormal,1691
7,CVC - Borderline,4086
8,CVC - Normal,13041


In [71]:
fig = px.bar(categoryCounts, x = 'Number of Observations', y = 'Malpositions', template = 'plotly_white', title = 'Line and tube positions')
fig.show()

In [72]:
countsDf = {'Position Type' : [], 'Malposition'  : [], 'Number of Observations' : []}

for Type in ['Normal','Abnormal','Borderline']:
    for malposition in ['ETT','NGT','CVC']:
        colName = f'{malposition} - {Type}'
        countsDf['Position Type'].append(Type)
        countsDf['Malposition'].append(malposition)
        val = df[colName].sum(axis = 0)
        countsDf['Number of Observations'].append(val)
countsDf = pd.DataFrame(countsDf)
countsDf

,Position Type,Malposition,Number of Observations
0,Normal,ETT,195
1,Normal,NGT,97
2,Normal,CVC,13041
3,Abnormal,ETT,3
4,Abnormal,NGT,17
5,Abnormal,CVC,1691
6,Borderline,ETT,25
7,Borderline,NGT,25
8,Borderline,CVC,4086


In [73]:
fig = px.bar(countsDf, x = 'Number of Observations', y = 'Position Type', color = 'Malposition', barmode = 'stack', template = 'plotly_white')
fig.show()

In [74]:
# Calculate the total number of observations for all position types
total_observations_all_types = countsDf['Number of Observations'].sum()

# Calculate the percentage for each position type
countsDf['Percentage'] = (countsDf['Number of Observations'] / total_observations_all_types) * 100

# Print the updated DataFrame with percentages
countsDf

,Position Type,Malposition,Number of Observations,Percentage
0,Normal,ETT,195,1.016684
1,Normal,NGT,97,0.505735
2,Normal,CVC,13041,67.992701
3,Abnormal,ETT,3,0.015641
4,Abnormal,NGT,17,0.088634
5,Abnormal,CVC,1691,8.816475
6,Borderline,ETT,25,0.130344
7,Borderline,NGT,25,0.130344
8,Borderline,CVC,4086,21.303441


In [75]:
# Assuming your DataFrame is named df
columns_to_drop = ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Normal']
df = df.drop(columns=columns_to_drop)
df

,StudyInstanceUID,CVC - Abnormal,CVC - Borderline,CVC - Normal,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,ec89415d1
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,1,0,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,1,0,0,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,1,207685cd1
8,1.2.826.0.1.3680043.8.498.91266853519846682206...,0,0,1,2edf3dce3
...,...,...,...,...,...
30075,1.2.826.0.1.3680043.8.498.12112840402677606176...,0,1,0,cccbc15ba
30076,1.2.826.0.1.3680043.8.498.59704742952729813362...,1,0,0,172c3c7ed
30077,1.2.826.0.1.3680043.8.498.97304417279653947772...,0,1,0,b304abf90
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,0,0,1,7192404d8


In [76]:
# Assuming filtered_df contains the filtered DataFrame after dropping rows with num_ones > 1

# Define the category columns you want to balance
category_columns = ['CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal']

# Get the count of observations in "ETT - Abnormal"
class_count = df['CVC - Abnormal'].sum()

# Create empty lists to store the sampled dataframes
sampled_dfs = []

# Sample the data for each category
for category in category_columns:
    if category != 'CVC - Abnormal':
        category_df = df[df[category] == 1]
        sampled_category_df = category_df.sample(class_count, replace=True, random_state=42)
        sampled_dfs.append(sampled_category_df)

# Include the original ETT - Abnormal rows
ett_abnormal_df = df[df['CVC - Abnormal'] == 1]
sampled_dfs.append(ett_abnormal_df)

# Concatenate the undersampled dataframes
undersampled_df = pd.concat(sampled_dfs, ignore_index=True)

# Shuffle the undersampled DataFrame
undersampled_df = undersampled_df.sample(frac=1, random_state=42)

# Print the count for each category
for category in category_columns:
    count = undersampled_df[category].sum()
    print(f"{category}: {count}")


CVC - Abnormal: 1691
CVC - Borderline: 1691
CVC - Normal: 1691


In [100]:
from PIL import Image
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

# Set the path to your image dataset directory
dataset_path = '/content/drive/My Drive/ranzcr-clip-catheter-line-classification/train'

# Get the StudyInstanceUID values from the undersampled_df
undersampled_study_ids = undersampled_df['StudyInstanceUID'].head(100).tolist()

# Initialize lists to store image names and resized pixel arrays
StudyInstanceUID = []
resized_pixel_arrays = []

# Set the desired image size
desired_size = (2048, 2500)

# Loop through the image files, resize, and store resized pixel arrays for undersampled StudyInstanceUIDs
for study_id in tqdm(undersampled_study_ids):  # tqdm adds a progress bar
    image_file = study_id + '.jpg'  # Assuming the image filenames are StudyInstanceUID.jpg
    image_path = os.path.join(dataset_path, image_file)
    StudyInstanceUID.append(image_file)

    img = Image.open(image_path)
    resized_img = img.resize(desired_size)
    resized_img_array = np.array(resized_img)
    resized_pixel_arrays.append(resized_img_array)

# Create a pandas DataFrame with StudyInstanceUID and resized pixel arrays
resized_data = {'StudyInstanceUID': StudyInstanceUID, 'ResizedPixelArray': resized_pixel_arrays}
resized_imgdf = pd.DataFrame(resized_data)

resized_imgdf.head()

100%|██████████| 100/100 [00:46<00:00,  2.17it/s]


,StudyInstanceUID,ResizedPixelArray
0,1.2.826.0.1.3680043.8.498.10061097429791858965...,"[[253, 252, 252, 250, 249, 249, 249, 250, 251,..."
1,1.2.826.0.1.3680043.8.498.85805122814726704061...,"[[0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2,..."
2,1.2.826.0.1.3680043.8.498.10576118037679113226...,"[[38, 36, 31, 26, 23, 19, 16, 14, 11, 9, 7, 6,..."
3,1.2.826.0.1.3680043.8.498.83005605524452711368...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,1.2.826.0.1.3680043.8.498.43357590859554557620...,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2,..."


In [101]:
# Remove the ".jpg" extension from the StudyInstanceUID column in imgdf
resized_imgdf['StudyInstanceUID'] = resized_imgdf['StudyInstanceUID'].str.replace('.jpg', '')

# Merge the two dataframes on StudyInstanceUID
merged_df = pd.merge(undersampled_df, resized_imgdf, on='StudyInstanceUID')

# Add the ".jpg" extension back to the StudyInstanceUID column in imgdf
resized_imgdf['StudyInstanceUID'] = resized_imgdf['StudyInstanceUID'] + '.jpg'

<ipython-input-101-43944d3d2960>:2: FutureWarning:

The default value of regex will change from True to False in a future version.



In [79]:
merged_df.head()

,StudyInstanceUID,CVC - Abnormal,CVC - Borderline,CVC - Normal,PatientID,ResizedPixelArray
0,1.2.826.0.1.3680043.8.498.10061097429791858965...,0,0,1,b65a62423,"[[253, 252, 252, 250, 249, 249, 249, 250, 251,..."
1,1.2.826.0.1.3680043.8.498.85805122814726704061...,0,0,1,97f61561b,"[[0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2,..."
2,1.2.826.0.1.3680043.8.498.10576118037679113226...,0,1,0,95c6fe01c,"[[38, 36, 31, 26, 23, 19, 16, 14, 11, 9, 7, 6,..."
3,1.2.826.0.1.3680043.8.498.83005605524452711368...,1,0,0,929ca58b4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,1.2.826.0.1.3680043.8.498.43357590859554557620...,0,0,1,50d613170,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2,..."


In [102]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

merged_df.shape

(119, 6)

In [103]:
from pandas.core.generic import RandomState
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

X = merged_df.drop("CVC - Abnormal", axis=1)
Y = merged_df["CVC - Abnormal"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42, test_size=0.2)

X_train = np.array([np.resize(arr, (28, 28)) for arr in X_train["ResizedPixelArray"].tolist()])
X_test = np.array([np.resize(arr, (28, 28)) for arr in X_test["ResizedPixelArray"].tolist()])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=10)

test_loss, test_acc = model.evaluate(X_test, Y_test, verbose=2)

print("Test accuracy:", test_acc)

# probability_model = tf.keras.Sequential([model,
#                                          tf.keras.layers.Softmax()])

# predictions = probability_model.predict(Y_train)



Epoch 1/10
3/3 [==============================] - 1s 7ms/step - loss: 34.0004 - accuracy: 0.3789
Epoch 2/10
3/3 [==============================] - 0s 7ms/step - loss: 42.2217 - accuracy: 0.6737
Epoch 3/10
3/3 [==============================] - 0s 7ms/step - loss: 27.0518 - accuracy: 0.5053
Epoch 4/10
3/3 [==============================] - 0s 6ms/step - loss: 12.9562 - accuracy: 0.5895
Epoch 5/10
3/3 [==============================] - 0s 7ms/step - loss: 32.2513 - accuracy: 0.6632
Epoch 6/10
3/3 [==============================] - 0s 7ms/step - loss: 16.6610 - accuracy: 0.6316
Epoch 7/10
3/3 [==============================] - 0s 12ms/step - loss: 15.9669 - accuracy: 0.4632
Epoch 8/10
3/3 [==============================] - 0s 7ms/step - loss: 13.3381 - accuracy: 0.6316
Epoch 9/10
3/3 [==============================] - 0s 6ms/step - loss: 6.9981 - accuracy: 0.6632
Epoch 10/10
3/3 [==============================] - 0s 6ms/step - loss: 5.8617 - accuracy: 0.6526
1/1 - 0s - loss: 4.1469 - accu